# Getting Started

## Install Libraries

In [38]:
!pip install fire

In [39]:
from google.colab import auth
auth.authenticate_user(project_id='csci8953')

import pandas
import numpy
from osgeo import gdal #for command line tools: !apt install gdal-bin
import matplotlib
import scipy
import tqdm
import fire
import torch
from google.colab import drive
import os
from pathlib import Path

# Specify the path you want to check
PATH_PYTHON = '/content/drive/MyDrive/CSCI-8523-AI-For-Earth-Group-Folder/PythonFiles/'
PATH_DATA = '/content/drive/MyDrive/CSCI-8523-AI-For-Earth-Group-Folder/Data/'
PATH_PROJECT = '/content/drive/MyDrive/CSCI-8523-AI-For-Earth-Group-Folder/'
PATH_YIELD_DATA = '/content/drive/MyDrive/CSCI-8523-AI-For-Earth-Group-Folder/Data/USDA/yield_data.csv'

## Determine file-path for exporting data from Earth Engine.

In [40]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# List folders in the specified path
folders = [f for f in os.listdir(PATH_PYTHON) if os.path.isdir(os.path.join(PATH_PYTHON, f))]
print(folders)

['cyp']


In [42]:
# Change the current working directory to your folder
os.chdir(PATH_PROJECT)

# Confirm the current working directory
print("Current Directory:", os.getcwd())

Current Directory: /content/drive/.shortcut-targets-by-id/11g4MHwwyePEqWkjhQqw7esTFupZe7lgt/CSCI-8523-AI-For-Earth-Group-Folder


## Check to ensure Earth Engine API is functioning correctly.

In [43]:
import ee
ee.Authenticate()
ee.Initialize()

#Export the Data

In [44]:
#%cd /content/drive/MyDrive/CSCI-8523-AI-For-Earth-Group-Folder/PythonFiles/
%cd /content/drive/MyDrive/CSCI-8523-Data/

/content/drive/MyDrive/CSCI-8523-Data


In [45]:
%pwd

'/content/drive/MyDrive/CSCI-8523-Data'

In [46]:
folders = [f for f in os.listdir(PATH_PYTHON) if os.path.isdir(os.path.join(PATH_PYTHON, f))]
print(folders)

['cyp']


Currently trying to determine how we can export files to specific folder in Google Drive.

Update: Solved! Was using a deprecated version of MODIS dataset. I.e. "MODIS/MOD09A1" when I should have used "MODIS/061/MOD09A1".

## Export Data

In [ ]:
!python run.py export

## Preprocess Data

In [ ]:
!python run.py process

## Feature Engineering

In [ ]:
!python run.py engineer

## Train Transformer

In [ ]:
!python run.py train_transformer_encoder

# Visualization

In [64]:
import torch
from bs4 import BeautifulSoup
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt


def plot_county_errors(model, svg_file, save_colorbar=True):
    """
    For the most part, reformatting of
    https://github.com/JiaxuanYou/crop_yield_prediction/blob/master/6%20result_analysis/yield_map.py

    Generates an svg of the counties, coloured by their prediction error.

    Parameters
    ----------
    model: pathlib Path
        Path to the model being plotted.
    svg_file: pathlib Path, default=Path('data/counties.svg')
        Path to the counties svg file used as a base
    save_colorbar: boolean, default=True
        Whether to save a colorbar too.
    """

    model_sd = torch.load(model, map_location="cpu")

    print(model.parents)

    model_dir = model.parents[0]

    real_values = model_sd["test_real"]
    pred_values = model_sd["test_pred"]

    gp = True
    try:
        gp_values = model_sd["test_pred_gp"]
    except KeyError:
        gp = False

    indices = model_sd["test_indices"]

    pred_err = pred_values - real_values
    pred_dict = {}
    for idx, err in zip(indices, pred_err):
        state, county = idx

        state = str(state).zfill(2)
        county = str(county).zfill(3)

        pred_dict[state + county] = err

    model_info = model.name[:-8].split("_")

    colors = [
        "#b2182b",
        "#d6604d",
        "#f4a582",
        "#fddbc7",
        "#d1e5f0",
        "#92c5de",
        "#4393c3",
        "#2166ac",
    ]

    _single_plot(
        pred_dict, svg_file, model_dir / f"{model_info[0]}_{model_info[1]}.svg", colors
    )

    if gp:
        gp_pred_err = gp_values - real_values
        gp_dict = {}
        for idx, err in zip(indices, gp_pred_err):
            state, county = idx

            state = str(state).zfill(2)
            county = str(county).zfill(3)

            gp_dict[state + county] = err

    _single_plot(
        gp_dict, svg_file, model_dir / f"{model_info[0]}_{model_info[1]}_gp.svg", colors
    )

    if save_colorbar:
        _save_colorbar(model_dir / "colorbar.png", colors)


def _single_plot(err_dict, svg_file, savepath, colors):

    # load the svg file
    svg = svg_file.open("r").read()
    # Load into Beautiful Soup
    soup = BeautifulSoup(svg, features="html.parser")
    # Find counties
    paths = soup.findAll("path")

    path_style = (
        "font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1"
        ";stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start"
        ":none;stroke-linejoin:bevel;fill:"
    )

    for p in paths:
        if p["id"] not in ["State_Lines", "separator"]:
            try:
                rate = err_dict[p["id"]]
            except KeyError:
                continue
            if rate > 15:
                color_class = 7
            elif rate > 10:
                color_class = 6
            elif rate > 5:
                color_class = 5
            elif rate > 0:
                color_class = 4
            elif rate > -5:
                color_class = 3
            elif rate > -10:
                color_class = 2
            elif rate > -15:
                color_class = 1
            else:
                color_class = 0

            color = colors[color_class]
            p["style"] = path_style + color
    soup = soup.prettify()
    with savepath.open("w") as f:
        f.write(soup)


def _save_colorbar(savedir, colors):
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.02, 0.8])

    cmap = mpl.colors.ListedColormap(colors[1:-1])

    cmap.set_over(colors[-1])
    cmap.set_under(colors[0])

    bounds = [-15, -10, -5, 0, 5, 10, 15]

    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    cb = mpl.colorbar.ColorbarBase(
        ax,
        cmap=cmap,
        norm=norm,
        # to use 'extend', you must
        # specify two extra boundaries:
        boundaries=[-20] + bounds + [20],
        extend="both",
        ticks=bounds,  # optional
        spacing="proportional",
        orientation="vertical",
    )
    plt.savefig(savedir, dpi=300, bbox_inches="tight")

In [71]:
plot_county_errors(Path("/content/drive/MyDrive/CSCI-8523-Data/Data/Second Best Transformer/Transformer_2009_5_32_.pth.tar"), Path("/content/drive/MyDrive/CSCI-8523-Data/Data/counties.svg"))

<ipython-input-64-545adf487aea>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_sd = torch.load(model, map_location="cpu")


<PosixPath.parents>


UnboundLocalError: local variable 'gp_dict' referenced before assignment